In [1]:

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
model_path = "conf3-lstm-model.h5"
model = load_model(model_path)
n_seq = 30

In [3]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

In [4]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

In [5]:
from jaad_data import JAAD

In [6]:
imdb = JAAD(data_path = jaad_path)

In [18]:
def play():
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print( length )
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0
    pred = True

    # Capture frame-by-frame
    ret = True
    
    while(ret and i <  frames[0]):
        ret, origimg = cap.read()
        i += 1
    print(i)
    i = 0
    while(ret and i < lstm_seq.shape[0] - n_seq -1):
        ret, origimg = cap.read()
        batch = lstm_seq[i:i + n_seq]
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False ):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            #batch =  np.delete(batch, 0, axis = 0)
            #batch =  np.append(batch, y_hat, axis = 0)
            print(len(batch))
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
           
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)

            if pred:
                xmin = int(inv_yhat[0] / 2)
                ymin = int(inv_yhat[1] / 2)
                xmax = int(inv_yhat[2] / 2)
                ymax = int(inv_yhat[3] / 2)
            else:
                xmin = int(bb_cross[ped_b[ped_index]][i][0] / 2)
                ymin = int(bb_cross[ped_b[ped_index]][i][1] / 2)
                xmax = int(bb_cross[ped_b[ped_index]][i][2] / 2)
                ymax = int(bb_cross[ped_b[ped_index]][i][3] / 2)

            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)

            # Draw the predicted boxes in blue
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            xmin_ = int (df[i:i+1][0] / 2)
            ymin_ = int (df[i:i+1][1] / 2)
            xmax_ = int (df[i:i+1][2] / 2)
            ymax_ = int (df[i:i+1][3] / 2)
            
            # Draw the present boxes in green
            cv2.rectangle(img, (xmin_, ymin_), (xmax_, ymax_ ), (0, 255, 0), 1)
            
            # Draw the actual future boxes in red
            cv2.rectangle(img, (int (df[i+30:i+31][0] / 2), int (df[i+30:i+31][1] / 2)),
                          (int (df[i+30:i+31][2] / 2), int (df[i+30:i+31][3] / 2) ), (0, 0,255), 1)


            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(33) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [22]:
vid = "video_0318"

In [23]:
anno = imdb._get_annotations(vid)

In [24]:
anno['width']

1920

In [25]:
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

for i, p in enumerate(ped_b):
    #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
        print(p)
print(ped_b)
#for k,v in bb_cross.items():
#    print(k)

0_318_2542b
['0_318_2542b']


In [26]:
ped_index = 0
lstm_seq =  bb_cross[ped_b[ped_index]]
frames = (anno['ped_annotations'][ped_b[ped_index]]['frames'][0], len(anno['ped_annotations'][ped_b[ped_index]]['frames']))

In [27]:
#frames
#len(lstm_seq)
#l = [k for k,v in anno['ped_annotations'][ped_b[ped_index]].items()]

In [28]:
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

for i in range(df.shape[0]):
    xmin_ = int (df[i:i+1][0] / 2)
    ymin_ = int (df[i:i+1][1] / 2)
    xmax_ = int (df[i:i+1][2] / 2)
    ymax_ = int (df[i:i+1][3] / 2)
    #print(i, xmin_, ymin_, xmax_, ymax_)

In [29]:
pickle_in = open("min-max-scaler.pkl","rb")
scaler = pickle.load(pickle_in)
#print(type(scaler))
#print(scaler.data_max_)
#print(scaler.data_min_)

lstm_seq = scaler.transform(lstm_seq)
#batch = scaler.transform(df)

In [30]:
#lstm_seq.shape[0]
#frames[1]

In [31]:
play()

240
0
[[0.35 0.63 0.43 0.86]]
30
[661.22 596.21 832.19 927.62]
(330, 298)
(416, 463)
[[0.35 0.63 0.43 0.86]]
30
[661.7  599.33 833.18 930.29]
(330, 299)
(416, 465)
[[0.35 0.63 0.43 0.86]]
30
[660.17 601.2  832.04 931.85]
(330, 300)
(416, 465)
[[0.35 0.63 0.43 0.86]]
30
[661.16 598.91 833.04 929.58]
(330, 299)
(416, 464)
[[0.35 0.63 0.43 0.86]]
30
[660.87 596.89 833.3  927.58]
(330, 298)
(416, 463)
[[0.35 0.63 0.44 0.86]]
30
[660.71 597.32 834.82 928.  ]
(330, 298)
(417, 464)
[[0.35 0.63 0.43 0.86]]
30
[661.03 596.51 834.7  927.09]
(330, 298)
(417, 463)
[[0.35 0.62 0.43 0.86]]
30
[660.88 595.2  834.51 925.78]
(330, 297)
(417, 462)
[[0.35 0.62 0.43 0.86]]
30
[660.87 594.14 834.53 924.74]
(330, 297)
(417, 462)
[[0.35 0.62 0.44 0.86]]
30
[660.45 591.17 836.21 928.06]
(330, 295)
(418, 464)
[[0.35 0.62 0.44 0.86]]
30
[659.44 589.63 835.96 929.17]
(329, 294)
(417, 464)
[[0.35 0.62 0.44 0.86]]
30
[659.92 587.43 839.16 931.26]
(329, 293)
(419, 465)
[[0.35 0.61 0.44 0.86]]
30
[660.27 584.21 840.

[[0.37 0.59 0.5  0.95]]
30
[ 712.43  559.73  959.11 1024.12]
(356, 279)
(479, 512)
[[0.38 0.59 0.5  0.95]]
30
[ 714.15  558.55  960.8  1021.41]
(357, 279)
(480, 510)
[[0.38 0.59 0.5  0.95]]
30
[ 712.92  558.59  959.82 1021.98]
(356, 279)
(479, 510)
[[0.38 0.59 0.5  0.95]]
30
[ 715.51  557.82  963.51 1019.85]
(357, 278)
(481, 509)
[[0.38 0.58 0.5  0.94]]
30
[ 714.7   556.42  962.28 1017.22]
(357, 278)
(481, 508)
[[0.38 0.58 0.5  0.94]]
30
[ 715.92  556.74  963.44 1016.65]
(357, 278)
(481, 508)
[[0.38 0.58 0.5  0.96]]
30
[ 713.17  553.82  962.45 1031.67]
(356, 276)
(481, 515)
[[0.38 0.59 0.5  0.95]]
30
[ 716.78  557.51  964.07 1028.01]
(358, 278)
(482, 514)
[[0.38 0.58 0.5  0.95]]
30
[ 715.67  556.57  964.28 1024.32]
(357, 278)
(482, 512)
[[0.38 0.58 0.5  0.95]]
30
[ 716.54  555.67  965.34 1025.35]
(358, 277)
(482, 512)
[[0.38 0.58 0.5  0.95]]
30
[ 716.61  556.02  966.64 1024.07]
(358, 278)
(483, 512)
[[0.38 0.58 0.5  0.95]]
30
[ 716.3   556.06  966.11 1022.73]
(358, 278)
(483, 511)
[[0.

[[0.32 0.61 0.43 0.95]]
30
[ 605.71  584.75  833.59 1022.91]
(302, 292)
(416, 511)
[[0.32 0.62 0.43 0.95]]
30
[ 602.67  587.54  831.22 1023.13]
(301, 293)
(415, 511)
[[0.31 0.62 0.43 0.95]]
30
[ 596.86  589.04  826.35 1024.32]
(298, 294)
(413, 512)
[[0.31 0.62 0.43 0.95]]
30
[ 593.55  592.59  823.82 1025.35]
(296, 296)
(411, 512)
[[0.31 0.62 0.43 0.95]]
30
[ 587.93  594.35  817.34 1026.39]
(293, 297)
(408, 513)
[[0.31 0.63 0.42 0.95]]
30
[ 583.21  596.25  813.49 1025.92]
(291, 298)
(406, 512)
No of frames: 209
Total Time: 3.622631072998047
fps: 57.69287454022583
